In [2]:
import math
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import datetime
import dateutil.parser
import math
import plotly.express as px
import dateutil.relativedelta
from datetime import datetime


def create_df(df,steps=1):
    
    dataX,dataY = [],[]
    
    for i in range(len(df)-steps-1):
        a = df[i:(i+steps),0]
        dataX.append(a)
        dataY.append(df[i+steps,0])
    return np.array(dataX),np.array(dataY)


def prever_valor_lstm(array,dias_anteriores,dias_previsao,epocas):

    if len(array) < 130:
        dias_anteriores = 10
        
    
    dias_retorno = dias_anteriores
    
    dias_previsao = dias_previsao
    

    qtd_linhas = len(array)

    qtd_linhas_treino = round(.70*qtd_linhas)


    qtd_linhas_teste = qtd_linhas-qtd_linhas_treino

    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(array)

    #Separa em treino e teste

    train = df_scaled[:qtd_linhas_treino]
    test = df_scaled[qtd_linhas_treino:qtd_linhas_treino+qtd_linhas_teste]




    #gerando dados de treino e de teste

    steps = dias_retorno

    X_train,Y_train = create_df(train,steps)
    X_teste,Y_teste = create_df(test,steps)
    
    
    if(len(X_train)>0):

        #print('Treino x: ',X_train.shape,' Treino y: ',Y_train.shape)
        #print('Teste x: ',X_teste.shape,' Teste y: ',Y_teste.shape)

        """Esse 1 significa a quantidade de features o modelo tem"""
        X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
        X_teste = X_teste.reshape(X_teste.shape[0],X_teste.shape[1],1)




        #montando a rede
        model = Sequential()
        model.add(LSTM(35,return_sequences=True,input_shape=(steps,1)))
        model.add(LSTM(35,return_sequences=True))
        model.add(LSTM(35))
        model.add(Dropout(0.2))
        model.add(Dense(1))

        model.compile(optimizer='adam',loss='mse')
        #model.summary()

        validation = model.fit(X_train,Y_train,validation_data=(X_teste,Y_teste),epochs=epocas,batch_size=dias_retorno,verbose=2)
        
        length_test = len(test)

        days_input_steps = length_test - steps
        #print("Tamanho do test: ",length_test)
        #print("Quantidade de dias retornados: ",days_input_steps)

        input_steps = test[days_input_steps:]
        input_steps = np.array(input_steps).reshape(1,-1)
        #input_steps

        #Transformar em lista
        list_output_steps = list(input_steps)
        list_output_steps = list_output_steps[0].tolist()
        #list_output_steps

        pred_output =[]
        i = 0
        n_future = 1

        while(i<n_future):
            if(len(list_output_steps)>steps):
                input_steps = np.array(list_output_steps[1:])
                #print("{} dia. Valores de entrada -> {}".format(i,input_steps))
                input_steps = input_steps.reshape(1,-1)
                input_steps = input_steps.reshape((1,steps,1))
                pred = model.predict(input_steps,verbose=0)
                #print("{} dia. Valor previsto {}".format(i,pred))
                list_output_steps.extend(pred[0].tolist())
                list_output_steps = list_output_steps[1:]
                pred_output.extend(pred.tolist())
                i=i+1
            else:
                input_steps = input_steps.reshape((1,steps,1))
                pred = model.predict(input_steps, verbose=0)
                #print(pred[0])
                list_output_steps.extend(pred[0].tolist())
                #print(len(list_output_steps))
                pred_output.extend(pred.tolist())
                i=i+1

        #print(pred_output)

        prev = scaler.inverse_transform(pred_output)
        prev = np.array(prev).reshape(1,-1)
        #proxima_alta = list(prev)
        proxima_alta = prev[0].tolist()

    return proxima_alta    

In [3]:
ticker = "PRIO3.SA"
acao = yf.download(ticker)


df = pd.DataFrame(acao[:"2022-11-21"])


[*********************100%***********************]  1 of 1 completed


In [5]:
alta = prever_valor_lstm(pd.DataFrame(df["High"]),4,1,100)
baixa = prever_valor_lstm(pd.DataFrame(df["Low"]),4,1,100)

Epoch 1/100
523/523 - 10s - loss: 0.0780 - val_loss: 0.0144 - 10s/epoch - 19ms/step
Epoch 2/100
523/523 - 3s - loss: 0.0166 - val_loss: 0.0106 - 3s/epoch - 6ms/step
Epoch 3/100
523/523 - 3s - loss: 0.0186 - val_loss: 0.0173 - 3s/epoch - 6ms/step
Epoch 4/100
523/523 - 3s - loss: 0.0138 - val_loss: 0.0170 - 3s/epoch - 6ms/step
Epoch 5/100
523/523 - 3s - loss: 0.0156 - val_loss: 0.0098 - 3s/epoch - 6ms/step
Epoch 6/100
523/523 - 3s - loss: 0.0150 - val_loss: 0.0083 - 3s/epoch - 6ms/step
Epoch 7/100
523/523 - 3s - loss: 0.0138 - val_loss: 0.0083 - 3s/epoch - 6ms/step
Epoch 8/100
523/523 - 3s - loss: 0.0154 - val_loss: 0.0263 - 3s/epoch - 6ms/step
Epoch 9/100
523/523 - 3s - loss: 0.0141 - val_loss: 0.0137 - 3s/epoch - 6ms/step
Epoch 10/100
523/523 - 3s - loss: 0.0144 - val_loss: 0.0077 - 3s/epoch - 6ms/step
Epoch 11/100
523/523 - 3s - loss: 0.0123 - val_loss: 0.0077 - 3s/epoch - 6ms/step
Epoch 12/100
523/523 - 3s - loss: 0.0125 - val_loss: 0.0149 - 3s/epoch - 6ms/step
Epoch 13/100
523/523 -

Epoch 1/100
523/523 - 10s - loss: 0.0613 - val_loss: 0.0166 - 10s/epoch - 18ms/step
Epoch 2/100
523/523 - 3s - loss: 0.0179 - val_loss: 0.0221 - 3s/epoch - 6ms/step
Epoch 3/100
523/523 - 3s - loss: 0.0167 - val_loss: 0.0172 - 3s/epoch - 6ms/step
Epoch 4/100
523/523 - 3s - loss: 0.0155 - val_loss: 0.0100 - 3s/epoch - 6ms/step
Epoch 5/100
523/523 - 3s - loss: 0.0166 - val_loss: 0.0109 - 3s/epoch - 6ms/step
Epoch 6/100
523/523 - 3s - loss: 0.0145 - val_loss: 0.0193 - 3s/epoch - 6ms/step
Epoch 7/100
523/523 - 3s - loss: 0.0150 - val_loss: 0.0131 - 3s/epoch - 6ms/step
Epoch 8/100
523/523 - 3s - loss: 0.0143 - val_loss: 0.0146 - 3s/epoch - 6ms/step
Epoch 9/100
523/523 - 3s - loss: 0.0118 - val_loss: 0.0133 - 3s/epoch - 6ms/step
Epoch 10/100
523/523 - 3s - loss: 0.0135 - val_loss: 0.0175 - 3s/epoch - 6ms/step
Epoch 11/100
523/523 - 3s - loss: 0.0165 - val_loss: 0.0081 - 3s/epoch - 6ms/step
Epoch 12/100
523/523 - 3s - loss: 0.0129 - val_loss: 0.0075 - 3s/epoch - 6ms/step
Epoch 13/100
523/523 -

In [11]:
acao.tail(2)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-11-22 00:00:00-03:00,34.959999,35.990002,34.830002,35.470001,35.470001,13516800
2022-11-23 00:00:00-03:00,34.980000,35.250000,34.740002,35.130001,35.130001,510700


In [6]:
print(alta,baixa)

[33.41595863879792] [32.62994501544941]


In [14]:
df["High"]

Date
2010-10-25 00:00:00-02:00    24.400000
2010-10-26 00:00:00-02:00    23.400000
2010-10-27 00:00:00-02:00    23.200001
2010-10-28 00:00:00-02:00    22.709999
2010-10-29 00:00:00-02:00    22.900000
                               ...    
2022-11-11 00:00:00-03:00    38.299999
2022-11-14 00:00:00-03:00    39.250000
2022-11-16 00:00:00-03:00    38.650002
2022-11-17 00:00:00-03:00    35.799999
2022-11-18 00:00:00-03:00    36.450001
Name: High, Length: 2995, dtype: float64

In [13]:
acao

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-10-25 00:00:00-02:00,24.100000,24.400000,22.100800,23.340000,23.340000,25180000
2010-10-26 00:00:00-02:00,22.800200,23.400000,22.500000,22.500000,22.500000,7200000
2010-10-27 00:00:00-02:00,22.500000,23.200001,22.202400,22.700001,22.700001,3685000
2010-10-28 00:00:00-02:00,22.709999,22.709999,22.100000,22.500000,22.500000,2900000
2010-10-29 00:00:00-02:00,22.600000,22.900000,22.400000,22.740000,22.740000,825000
...,...,...,...,...,...,...
2022-11-16 00:00:00-03:00,38.570000,38.650002,35.340000,36.270000,36.270000,20821900
2022-11-17 00:00:00-03:00,35.209999,35.799999,34.200001,35.799999,35.799999,21667300
2022-11-18 00:00:00-03:00,36.099998,36.450001,34.250000,34.500000,34.500000,18514800
